<a href="https://colab.research.google.com/github/brendarlq/llm-sarcasmo/blob/main/Prueba_con_OPENAPI_para_deteccion_de_sarcasmo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================
# 🧩 CONFIGURACIÓN INICIAL
# ==============================
!pip install openai datasets scikit-learn tqdm

import os
from openai import OpenAI
import pandas as pd

# ==============================
# 🔑 API KEY
# ==============================
import getpass

from openai import OpenAI

client = OpenAI()


# ==============================
# 🤖 FUNCIÓN DE DETECCIÓN DE SARCASMO (LLM)
# ==============================
def detect_sarcasm_zero_shot(text):
    prompt = f"""
    Determina si el siguiente texto es sarcástico o no.
    Responde únicamente con JSON válido que contenga:
    {{
      "sarcastic": 1 o 0,
      "explanation": "breve explicación en español"
    }}

    Texto: "{text}"
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        content = response.choices[0].message.content.strip()
        if content.startswith("```json"):
            content = content.replace("```json", "").replace("```", "").strip()
        return content
    except Exception as e:
        return f'{{"sarcastic": -1, "explanation": "error: {str(e)}"}}'

def detect_sarcasm_fewshot(text):
    prompt = f"""
    A continuación te mostraré ejemplos de textos sarcásticos y no sarcásticos.
    Después, deberás clasificar el texto nuevo siguiendo el mismo formato.

    Ejemplo 1:
    Texto: "Qué suerte que me enfermé justo el fin de semana."
    Respuesta: {{"sarcastic": 1, "explanation": "El hablante expresa ironía porque enfermarse no es algo afortunado."}}

    Ejemplo 2:
    Texto: "Hoy hace un día precioso, perfecto para pasear."
    Respuesta: {{"sarcastic": 0, "explanation": "Es una afirmación literal, sin ironía."}}

    Ejemplo 3:
    Texto: "Sí, claro, porque todo me sale bien últimamente..."
    Respuesta: {{"sarcastic": 1, "explanation": "El tono de resignación indica sarcasmo; el hablante dice lo contrario de lo que siente."}}

    Ahora analizá el siguiente texto y respondé con el mismo formato JSON:
    Texto: "{text}"
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # Podés cambiarlo a gpt-4o si querés más precisión
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        content = response.choices[0].message.content.strip()
        if content.startswith("```json"):
            content = content.replace("```json", "").replace("```", "").strip()
        return content
    except Exception as e:
        return f'{{"sarcastic": -1, "explanation": "error: {str(e)}"}}'


In [ ]:
# ==============================
# 🧪 PRUEBAS CON 10 FRASES + MÉTRICAS
# ==============================
import json
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

# Lista de frases y etiquetas reales (1 = sarcástico, 0 = no sarcástico)
frases_prueba = [
    ("Me encanta el calor 🫠", 1),                               # sarcasmo obvio
    ("Qué alegría, otro lunes más de trabajo interminable.", 1), # sarcasmo implícito
    ("Hoy sí que tuve suerte, se me rompió el auto otra vez.", 1), # sarcasmo negativo
    ("Nada mejor que estar atrapado en el tráfico una hora.", 1),  # sarcasmo irónico
    ("Estoy feliz de que mi computadora se haya colgado justo antes de entregar el informe.", 1), # sarcasmo resignado
    ("El cielo está despejado y hace una temperatura perfecta.", 0), # literal
    ("Amo cuando la gente llega tarde, me encanta esperar.", 1),     # sarcasmo clásico
    ("Por fin algo sale bien, ya era hora.", 0),                     # literal/ambigua
    ("Qué sorpresa, el colectivo llegó tarde otra vez.", 1),         # sarcasmo leve
    ("Me siento genial después de dormir solo dos horas.", 1)        # sarcasmo irónico
]

# Lista donde guardamos los resultados
resultados = []

print("⏳ Ejecutando detección de sarcasmo con ambos métodos...\n")

for frase, etiqueta_real in frases_prueba:
    zero = detect_sarcasm_zero_shot(frase)
    few  = detect_sarcasm_fewshot(frase)

    # Intentar parsear a JSON
    try:
        zero_json = json.loads(zero)
    except:
        zero_json = {"sarcastic": "?", "explanation": zero}
    try:
        few_json = json.loads(few)
    except:
        few_json = {"sarcastic": "?", "explanation": few}

    resultados.append({
        "Texto": frase,
        "Etiqueta Real": etiqueta_real,
        "Zero-Shot (Pred)": int(zero_json.get("sarcastic", -1)) if str(zero_json.get("sarcastic")).isdigit() else -1,
        "Zero-Shot (Explicación)": zero_json.get("explanation"),
        "Few-Shot (Pred)": int(few_json.get("sarcastic", -1)) if str(few_json.get("sarcastic")).isdigit() else -1,
        "Few-Shot (Explicación)": few_json.get("explanation")
    })

# Convertimos a DataFrame
df_resultados = pd.DataFrame(resultados)

# Mostrar tabla completa
pd.set_option("display.max_colwidth", None)
display(df_resultados)

# ==============================
# 📊 CÁLCULO DE MÉTRICAS
# ==============================

# Filtramos las filas válidas (sin errores)
df_valid = df_resultados[(df_resultados["Zero-Shot (Pred)"] != -1) & (df_resultados["Few-Shot (Pred)"] != -1)]

y_true = df_valid["Etiqueta Real"].astype(int)
y_zero = df_valid["Zero-Shot (Pred)"].astype(int)
y_few  = df_valid["Few-Shot (Pred)"].astype(int)

# Crear función de reporte compacto
def resumen_metricas(y_true, y_pred, metodo):
    print(f"\n📈 MÉTRICAS PARA {metodo}:")
    print(f"Accuracy : {accuracy_score(y_true, y_pred):.3f}")
    print(f"Precision: {precision_score(y_true, y_pred, zero_division=0):.3f}")
    print(f"Recall   : {recall_score(y_true, y_pred, zero_division=0):.3f}")
    print(f"F1-score : {f1_score(y_true, y_pred, zero_division=0):.3f}")

# Mostrar métricas para ambos métodos
resumen_metricas(y_true, y_zero, "Zero-Shot")
resumen_metricas(y_true, y_few, "Few-Shot")

# ==============================
# 💾 GUARDAR RESULTADOS
# ==============================
df_resultados.to_csv("resultados_sarcasmo_prueba.csv", index=False)
print("\n✅ Resultados guardados en 'resultados_sarcasmo_prueba.csv'")


⏳ Ejecutando detección de sarcasmo con ambos métodos...



,Texto,Etiqueta Real,Zero-Shot (Pred),Zero-Shot (Explicación),Few-Shot (Pred),Few-Shot (Explicación)
0,Me encanta el calor 🫠,1,1,"El uso del emoji y la expresión 'me encanta' en el contexto del calor sugiere que la persona no disfruta realmente del calor, lo que indica sarcasmo.",1,"El uso del emoji de sudor y la expresión 'me encanta' sugieren sarcasmo, ya que el calor extremo generalmente no es algo que a la gente le guste."
1,"Qué alegría, otro lunes más de trabajo interminable.",1,1,"El texto expresa una supuesta alegría por el lunes, pero el contexto sugiere que en realidad se siente frustración por el trabajo, lo que indica sarcasmo.",1,"El hablante expresa ironía al referirse a un lunes de trabajo, que generalmente no es motivo de alegría."
2,"Hoy sí que tuve suerte, se me rompió el auto otra vez.",1,1,"El hablante expresa que tuvo 'suerte' al romperse el auto, lo cual es una situación negativa, lo que indica sarcasmo.",1,"El hablante utiliza ironía al afirmar que tuvo suerte por la rotura del auto, lo cual es una situación negativa."
3,Nada mejor que estar atrapado en el tráfico una hora.,1,1,"El comentario sugiere que estar atrapado en el tráfico es algo positivo, lo cual es irónico y se entiende como sarcasmo.",1,"El hablante utiliza ironía al afirmar que estar atrapado en el tráfico es algo positivo, cuando en realidad es una experiencia frustrante."
4,Estoy feliz de que mi computadora se haya colgado justo antes de entregar el informe.,1,1,"El hablante expresa felicidad por una situación claramente negativa, lo que indica sarcasmo.",1,El hablante expresa ironía al decir que está 'feliz' por un problema que es frustrante y perjudicial.
5,El cielo está despejado y hace una temperatura perfecta.,0,0,"El texto parece ser una afirmación literal sobre el clima, sin indicios de ironía o burla.",0,"Es una afirmación literal sobre el clima, sin ironía."
6,"Amo cuando la gente llega tarde, me encanta esperar.",1,1,"El texto expresa una opinión positiva sobre algo que generalmente es considerado negativo, lo que indica sarcasmo.",1,"El hablante utiliza ironía al afirmar que le encanta esperar, ya que generalmente llegar tarde es considerado molesto."
7,"Por fin algo sale bien, ya era hora.",0,1,"El comentario sugiere que las cosas normalmente no salen bien, lo que implica un tono sarcástico al expresar alivio por un éxito que debería ser normal.",1,"El hablante utiliza un tono irónico, sugiriendo que las cosas no han salido bien en el pasado, lo que implica sarcasmo."
8,"Qué sorpresa, el colectivo llegó tarde otra vez.",1,1,"El comentario sugiere que la llegada tardía del colectivo es algo habitual y esperado, lo que implica ironía en la expresión de sorpresa.",1,"El hablante utiliza la palabra 'sorpresa' de manera irónica, ya que la tardanza del colectivo es algo esperado y no sorprendente."
9,Me siento genial después de dormir solo dos horas.,1,1,"El texto expresa una afirmación positiva sobre una situación que generalmente se considera negativa, lo que sugiere sarcasmo.",1,"El hablante utiliza ironía, ya que dormir solo dos horas generalmente no se asocia con sentirse genial."



📈 MÉTRICAS PARA Zero-Shot:
Accuracy : 0.900
Precision: 0.889
Recall   : 1.000
F1-score : 0.941

📈 MÉTRICAS PARA Few-Shot:
Accuracy : 0.900
Precision: 0.889
Recall   : 1.000
F1-score : 0.941

✅ Resultados guardados en 'resultados_sarcasmo_prueba.csv'


In [ ]:

# ==============================
# 💬 USANDO DATASET
# ==============================
"""
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import classification_report
import json

# Cargar dataset
dataset = load_dataset("Ernesto-1997/Sarcastic_spanish_dataset")
df = pd.DataFrame(dataset["train"])

TEXT_COL = "text"
LABEL_COL = "label"

# Muestrear 300 frases
df_sample = df.sample(300, random_state=42).reset_index(drop=True)

# Procesar por lotes
results = []
for i, row in tqdm(df_sample.iterrows(), total=len(df_sample)):
    res = detect_sarcasm(row[TEXT_COL])
    results.append(res)

df_sample["llm_raw"] = results

# Parsear resultados
def safe_parse(x):
    try:
        return json.loads(x)
    except:
        return {"sarcastic": -1, "explanation": "parse_error"}

parsed = df_sample["llm_raw"].apply(safe_parse)
df_sample["llm_pred"] = parsed.apply(lambda x: x.get("sarcastic", -1))
df_sample["llm_explanation"] = parsed.apply(lambda x: x.get("explanation", ""))

# Evaluar desempeño
y_true = df_sample[LABEL_COL].astype(int)
y_pred = df_sample["llm_pred"].astype(int)

print("\n📈 MÉTRICAS DEL LLM:")
print(classification_report(y_true, y_pred, digits=3))

# Guardar resultados
df_sample.to_csv("sarcasmo_llm_results.csv", index=False)
print("\n✅ Resultados guardados en 'sarcasmo_llm_results.csv'")
"""
